In [82]:
import boto3
import os
from dotenv import load_dotenv
from datetime import datetime
from botocore.exceptions import ClientError
import logging

load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")


class AwsS3():

    
    def upload_file(data : object, path : str, file_format) -> bool:

        """
            Upload a file to an S3 bucket
            :param file_name: File to upload
            :param bucket: Bucket to upload to
            :param object_name: S3 object name. If not specified then file_name is used
            :return: True if file was uploaded, else False
        """

        date = datetime.now().strftime("_%Y%m%d_%H%M%S")
        file_name = 'valorant_result{}{}'.format(date, file_format)
        input = path + file_name

        
        s3 = boto3.client("s3", aws_access_key_id = AWS_ACCESS_KEY_ID, aws_secret_access_key = AWS_SECRET_ACCESS_KEY)

        try:
            s3.put_object(Bucket = AWS_S3_BUCKET, Body = data, Key = input)

        except ClientError as e:
            logging.error(e)

            return False

        return True

    
    def get_file(path : str, file_name) -> str:

        """
            Get a file to an S3 bucket
            :param Path: Path to get
            :param bucket: Bucket to upload to
            :param object_name: S3 object name. If not specified then file_name is used
            :return: True if file was uploaded, else False
        """
        s3 = boto3.client('s3')
        
        try:

            response = s3.get_object(Bucket = AWS_S3_BUCKET, Key = file_name)
            data = response['Body'].read()
            data_str = data.decode('utf-8')

        except ClientError as e:
            logging.error(e)


        return data_str
        

    def get_files_list(path_read : str) -> list:

        s3 = boto3.resource('s3')
        bucket = s3.Bucket(AWS_S3_BUCKET)
        files_list = bucket.objects.filter(Prefix = path_read)
        files_list = list(files_list)
        if len(files_list) > 1:
            del files_list[0]
        else:
            pass

        return files_list

In [78]:
from aws_s3 import AwsS3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

class Crawler():
    
    def get_gun_report() -> str:

        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

        path_write = 'raw/trackergg/gun_report/'


            
        driver.get('https://api.tracker.gg/api/v2/valorant/standard/profile/riot/RayzenSama%236999/segments/weapon?playlist=competitive&seasonId=')
        
        data_pre = driver.find_element('xpath', '//pre').text
        file_format = '.txt'
        
        AwsS3.upload_file(data_pre, path_write, file_format)

        driver.quit()

        return data_pre

In [79]:
from aws_s3 import AwsS3
import pandas as pd
import json



class DataCleaner():
    
    def data_cleaner_trackergg():

        path_read = 'raw/trackergg/gun_report/'

        files = AwsS3.get_files_list(path_read)

        data_values = []
        
        for file in files:
            file = file.key

            data_s3 = AwsS3.get_file(path_read, file)

            data_json = json.loads(data_s3)

            weapons = data_json['data']

            

            for weapon in weapons:
                weapon_metadata = weapon["metadata"]
                weapon_stats = weapon["stats"]

                stat_dict = {}
                for stat, stat_data in weapon_stats.items():
                    stat_keys = weapon_stats.keys()
                    stat_columns = [f'{col}' for col in stat_keys]
                    stat_values = weapon_stats.values()
                    _stat_dict = {k: v for k, v in zip(stat_columns, stat_values)}
                    stat_dict.update(_stat_dict)
                
                row = {}
                row.update(weapon_metadata)
                row.update(stat_dict)

                data_values.append(row)

                    



        return data_values


        
    

In [ ]:
Crawler.get_gun_report()

In [83]:
data = DataCleaner.data_cleaner_trackergg()
data


[{'name': 'Ghost',
  'imageUrl': 'https://titles.trackercdn.com/valorant-api/weapons/1baa85b4-4c70-1284-64bb-6481dfc3bb4e/displayicon.png',
  'category': 'Sidearms',
  'matchesPlayed': {'rank': None,
   'percentile': None,
   'displayName': 'Matches',
   'displayCategory': 'Game',
   'category': 'game',
   'metadata': {},
   'value': 113,
   'displayValue': '113',
   'displayType': 'Number'},
  'matchesWon': {'rank': None,
   'percentile': None,
   'displayName': 'Wins',
   'displayCategory': 'Game',
   'category': 'game',
   'metadata': {},
   'value': 65,
   'displayValue': '65',
   'displayType': 'Number'},
  'matchesLost': {'rank': None,
   'percentile': None,
   'displayName': 'Losses',
   'displayCategory': 'Game',
   'category': 'game',
   'metadata': {},
   'value': 48,
   'displayValue': '48',
   'displayType': 'Number'},
  'matchesWinPct': {'rank': None,
   'percentile': None,
   'displayName': 'Win %',
   'displayCategory': 'Game',
   'category': 'game',
   'metadata': {},
 

In [84]:
df = pd.DataFrame(data)
# df_aux = pd.concat([df_aux, df], axis = 0)
df

,name,imageUrl,category,matchesPlayed,matchesWon,matchesLost,matchesWinPct,roundsPlayed,roundsWon,roundsLost,...,dealtBodyshots,dealtLegshots,receivedHeadshots,receivedBodyshots,receivedLegshots,mostKillsInMatch,mostKillsInRound,killDistance,avgKillDistance,longestKillDistance
0,Ghost,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...",...,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam..."
1,Classic,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...",...,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam..."
2,Melee,https://titles.trackercdn.com/valorant-api/wea...,Melee,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...",...,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam..."
3,Shorty,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...",...,"{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'rank': None, 'percentile': None, 'displayNam...","{'ran

In [85]:
df = pd.json_normalize(json.loads(json.dumps(data)))
df

# df.to_csv('guns.csv')

,name,imageUrl,category,matchesPlayed.rank,matchesPlayed.percentile,matchesPlayed.displayName,matchesPlayed.displayCategory,matchesPlayed.category,matchesPlayed.value,matchesPlayed.displayValue,...,avgKillDistance.displayValue,avgKillDistance.displayType,longestKillDistance.rank,longestKillDistance.percentile,longestKillDistance.displayName,longestKillDistance.displayCategory,longestKillDistance.category,longestKillDistance.value,longestKillDistance.displayValue,longestKillDistance.displayType
0,Ghost,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,None,None,Matches,Game,game,113,113,...,"1,549",Number,None,None,Longest Kill Distance,Combat,combat,4023,"4,023",Number
1,Classic,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,None,None,Matches,Game,game,180,180,...,"1,451",Number,None,None,Longest Kill Distance,Combat,combat,4046,"4,046",Number
2,Melee,https://titles.trackercdn.com/valorant-api/wea...,Melee,None,None,Matches,Game,game,4,4,...,107,Number,None,None,Longest Kill Distance,Combat,combat,200,200,Number
3,Shorty,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,None,None,Matches,Game,game,4,4,...,493,Number,None,None,Longest Kill Distance,Combat,combat,714,714,Number
4,Frenzy,https://titles.trackercdn.com/valorant-api/wea...,Sidearms,None,None,Matches,Game,game,18,18,...,"1,013",Number,None,None,Longest Kill Distance,Combat,combat,2573,"2,573",Number
5,Spectre,https://titles.trackercdn.com/valorant-api/wea...,SMGs,None,None,Matches,Game,game,175,175,...,"1,328",Number,None,None,Longest Kill Distance,Combat,combat,4979,"4,979",Number
6,Guardian,https://titles.trackercdn.com/valorant-api/wea...,Assault Rifles,None,None,Matches,Game,game,23,23,...,"2,747",Number,None,None,Longest Kill Distance,Combat,combat,4829,"4,829",Number
7,Ares,https://titles.trackercdn.com/valorant-api/wea...,Heavy Weapons,None,None,Matches,Game,game,64,64,...,"1,895",Number,None,None,Longest Kill Distance,Combat,combat,4409,"4,409",Number
8,Odin,https://titles.trackercdn.com/valorant-api/wea...,Heavy Weapons,None,None,Matches,Game,game,106,106,...,"1,931",Number,None,None,Longest Kill Distance,Combat,combat,4991,"4,991",Number
9,Bucky,https://titles.trackercdn.com/valorant-api/wea...,Shotguns,None,None,Matches,Game,game,7,7,...,785,Number,None,None,Longest Kill Distance,Combat,combat,1713,"1,713",Number
